# DZ locations normalizer

Thanks to Alexey Galda (GitHub: @alexgalda) and wingsuitworld.com for providing the raw CSV of drop zone locations and elevations where speeders and wingsuit pilots train and compete.


In [ ]:
import pandas as pd

## Get the raw data

In [ ]:
RAW_DATA_FILE = './resources/drop-zones-loc-elev-raw.csv'
rawData = pd.read_csv(RAW_DATA_FILE)

## Impute missing data and rename columns to SSScoring conventions

- Some DZ names are missing.  The city/town `name` will be imputed to the `DZ` column in those cases.
- Future revisions may use ISO-2 country codes instead of ISO-3.

In [ ]:
rawData.DZ = rawData.DZ.fillna(rawData.name)
data = pd.DataFrame(data = {
    'dropZone': rawData.DZ,
    'lat': rawData.lat,
    'lon': rawData.lon,
    'elevation': rawData.elevation,
    'location': rawData.name,
    'country': rawData.country,
})
data.sort_values(by='dropZone', ascending=True, inplace=True)
data.head(8)

## Preserve the normalized data

In [ ]:
data.to_csv('./ssscoring/resources/drop-zones-loc-elev.csv', sep=',')
!ls -l ./resources/drop-zones*

---
## Generate the synthetic Python module with the DZ data

This is necessary because Streamlit.io can't handle package resources natively for some reason or in a clean, Pythonic way.

In [11]:
dropZones = pd.read_csv('./ssscoring/resources/drop-zones-loc-elev.csv', sep=',')

In [12]:
dropZones.reset_index(drop=True)
dropZonesList = dropZones.to_dict('records')
for record in dropZonesList:
    del record['Unnamed: 0']

In [25]:
with open('./ssscoring/dzdir.py', 'w') as outputStream:
    outputStream.write('"""\nAUTO-GENERATED - do not modify by hand.\n"""\n\n')
    outputStream.write('DROP_ZONES_LIST = %s' % dropZonesList)
    outputStream.write('\n"""\n@private\n"""\n')
    outputStream.write('\n\n# end of auto-generated file')

### Confirm the dzdir module works

In [22]:
from ssscoring.dzdir import DROP_ZONES_LIST

In [19]:
x = pd.DataFrame(DROP_ZONES_LIST)

In [20]:
x

,dropZone,lat,lon,elevation,location,country
0,Aarhus,56.313258,10.6151104,20.0,Aarhus,DEN
1,Abbotsford,49.095333,-122.3131299,6.0,Abbotsford,CAN
2,Adrenalin Skydive,-34.809077,149.726989,653.0,Goulburn,AUS
3,Aero Parachute sport GmbH,51.408333,9.3775,277.0,Calden,GER
4,Aeroclub Menzelinsk,55.720724,53.060472,119.0,Menzelinsk,RUS
...,...,...,...,...,...,...
243,Vannes,47.723301,-2.71856,134.0,Vannes,FRA
244,Vatulino,55.662446,36.13871098,195.0,Vatulino,RUS
245,Vercelli,45.310803,8.42298925,125.0,Vercelli,ITA
246,Vesivehmaa,61.146440,25.69403887,153.0,Vesivehmaa,FIN
